# AOM Paper Matching

This is part of my effort in creating a dataset for studying the process of knowledge creation via scientists' collaboration, particularly the antecedents of collaboration performance. 

In order to measure collaboration performance, we start with using papers presented at the 2001-2014 Academy of Management (AOM) conferences. Because the key purpose of the conferences is for scientists to share their research ideas and on-going projects, the bar for paper acceptance at the AOM conferences is relatively lower than that at related top-tier journals. Therefore, I use AOM conference papers to capture scientists' collaboration and journal papers to capture successful collaboration. 

To identify successful collaboration, I am required to match AOM conference papers to journal papers. Matched AOM conference papers indicate successful collaborations. Unmatched AOM conference papers indicate unsuccesful collaborations.

In [ ]:
import os
import re
import sys
import csv
import pyprind
import threading

import numpy as np
import pandas as pd
import py_entitymatching as em

In [ ]:
# confirm Python & libraries version
print("python version: " + sys.version)
print("pandas version: " + pd.__version__)
print("magellan version: " + em.__version__)

python version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


## 1 - DATASET DESCRIPTION

2 datasets that we're using for matching

data is located at:

In [ ]:
path_to_csv_dir = '/content/drive/MyDrive/colab_notebooks/paper_matching/data'

### 1.A. AOM-conference-paper dataset
AOM-conference-paper dataset consists of the following information:
1. Conference paper ID 
2. Year of conference
3. Paper abstract 
4. Author list (last name, first name)
5. Author list (first name, last name)
6. First author's first name
7. First author's last name

Below is a snippet of the dataset

In [ ]:
CONF_INDEX = 'id'
CONF = ['id', 'year', 'bow', 'authors', 'authors_first_last', 'first_author_first', 'first_author_last']
CONF = em.read_csv_metadata(f"{path_to_csv_dir}/conferences_final_210607.csv", key=CONF_INDEX)
CONF.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,year,bow,authors,authors_first_last,first_author_first,first_author_last
0,2001_1001_1,2001,"variation,in,systemic,technologies,some,evidence,from,the,personal,computer,technological,system...","Ethiraj, Sendil",Sendil Ethiraj,Sendil,Ethiraj
1,2001_1001_2,2001,"looking,through,the,kaleidoscope,measuring,technological,capability,and,performance,does,the,dev...","Coombs, Joseph E.|Bierly, Paul",Joseph Coombs|Paul Bierly,Joseph,Coombs
2,2001_1001_3,2001,"the,concentration,of,basic,and,applied,research,in,the,semiconductor,and,pharmaceutical,industri...","Lim, Kwanghui",Kwanghui Lim,Kwanghui,Lim
3,2001_1001_4,2001,"my,strategy,says,see,you,in,court,determinants,of,decisions,not,to,settle,patent,litigation,in,c...","Somaya, Deepak",Deepak Somaya,Deepak,Somaya
4,2001_1001_5,2001,"departures,of,key,inventors,post,acquisitions,an,empirical,explanation,past,research,on,the,cons...","Paruchuri, Srikanth|Nerkar, Atul|Hambrick, Donald C.",Srikanth Paruchuri|Atul Nerkar|Donald Hambrick,Srikanth,Paruchuri


In the AOM conference-paper dataset, there are 37,587 rows (i.e. conference papers)

In [ ]:
CONF.shape[0]

37587

Below is the number of conference papers by conference year


In [ ]:
CONF.year.value_counts().sort_index()

2001    1129
2002    1427
2003    1950
2004    1982
2005    2477
2006    2895
2007    3285
2008    3514
2009    3358
2010    3159
2011    2869
2012    3374
2013    2882
2014    3286
Name: year, dtype: int64

### 1.B. Journal-paper dataset
Journal-paper dataset consists of the following information:
1. Publication ID 
2. Publication year
3. Publication abstract 
4. Author list (last name, first name)
5. Author list (first name, last name)

Below is a snippet of the dataset:

In [ ]:
JOUR_INDEX = 'id'
JOUR = ['id', 'year', 'bow', 'authors', 'authors_first_last']
JOUR = em.read_csv_metadata(f"{path_to_csv_dir}/journals_final_210607.csv", key=JOUR_INDEX)
JOUR.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,year,bow,authors,authors_first_last
0,1,2004,"a,rounded,picture,is,what,we,need,rhetorical,strategies,arguments,and,the,negotiation,of,change,...","Mueller, Frank|Sillince, John|Harvey, Charles|Howorth, Chris",Frank Mueller|John Sillince|Charles Harvey|Chris Howorth
1,2,2005,"an,indissoluble,mutual,destiny,the,north,staffordshire,potteries,and,the,limits,of,regional,trad...","Popp, Andrew",Andrew Popp
2,3,2009,"being,regimented,aspiration,discipline,and,identity,work,in,the,british,parachute,regiment,this,...","Thornborrow, Thomas|Brown, Andrew D.",Thomas Thornborrow|Andrew D. Brown
3,4,2016,"beyond,the,attitude,behaviour,gap,novel,perspectives,in,consumer,ethics,introduction,to,the,them...","Caruana, Robert|Carrington, Michal|Chatzidakis, Andreas",Robert Caruana|Michal Carrington|Andreas Chatzidakis
4,5,2015,"bouncing,back,from,a,loss,entrepreneurial,orientation,emotions,and,failure,narratives,in,this,st...","Wolfe, Marcus T.|Shepherd, Dean A.",Marcus T. Wolfe|Dean A. Shepherd


In the journal-paper dataset, there are 38,700 rows (i.e., journal papers)

In [ ]:
JOUR.shape[0]

38700

Below is the number of journal papers by year (from 2000)

In [ ]:
JOUR[JOUR.year>=2000].year.value_counts().sort_index()

2000     794
2001     751
2002     768
2003     797
2004     865
2005     903
2006     911
2007     976
2008    1035
2009    1210
2010    1106
2011    1040
2012     960
2013    1146
2014    1074
2015    1204
2016    1150
2017    1257
2018    1461
2019    1532
2020     439
Name: year, dtype: int64

## 2 - ENTITY MATCHING MODEL

Here we build our entity matching model 

A) Create new features for matching

B) Import a golden dataset

C) Train the entity matching models

D) Evaluate performance


### 2.A. Creation of New Features

In [ ]:
# import similarity measures
from py_stringmatching.similarity_measure.affine import Affine
from py_stringmatching.similarity_measure.tfidf import TfIdf


#### 2.A.1. Similarity of full authors list (using authors' names)

In [ ]:
def author_sim_affine(text1, text2):
    # default: (gap_start) gs = 1; (gap_continuation) gc = 0.5
    aff = Affine(gap_start=2, gap_continuation=0.5)
    js = text1.split('|')
    cs = text2.split('|')
    if len(js) < len(cs):
        denominator = len(js)
        left_set = js
        right_set = cs
    else:
        denominator = len(cs)
        left_set = cs
        right_set = js
    numerator = 0
    for i in left_set:
        score_max = -1
        for j in right_set:
            if score_max < aff.get_raw_score(j, i):
                score_max = aff.get_raw_score(j, i)
        numerator += score_max
    return numerator/denominator

def authors_similarity(ltuple, rtuple):
    j = str(ltuple["authors_first_last"])
    c = str(rtuple["authors_first_last"])
    return author_sim_affine(j, c)

#### 2.A.2. Similarity of first authors' names

In [ ]:
def first_author_similarity(ltuple, rtuple):
    c_first_author_first_last = str(rtuple['first_author_first']) + "_" +\
                                str(rtuple['first_author_last'])
    jauthors = str(ltuple["authors"])
    return author_sim_affine(c_first_author_first_last, jauthors)

#### 2.A.3. Similarity of paper abstracts

In [ ]:
def bow_tfidf(ltuple, rtuple):
    df = pd.read_csv(f'{path_to_csv_dir}/corpus.csv')
    tfidf = TfIdf(corpus_list=list(df.corpus))
    tfidf.set_dampen(True)
    bag1 = str(ltuple["bow"]).split(",")
    bag2 = str(rtuple["bow"]).split(",")
    return tfidf.get_sim_score(bag1, bag2)

#### 2.A.4. Difference between conference and publication years

In [ ]:
def year_difference(ltuple, rtuple):
    return ltuple["year"] - rtuple["year"]

### 2.B. Golden Data

The golden dataset was created following 3 steps:

<u><b>STEP 1 - Identify positive cases</u></b>
1. From the journal dataset, I first created a set of all journal papers (SET A) that were published between 2000 and 2014
2. For each journal paper in SET A, I identified a set of all conference papers that potentially matches with the focal journal paper (SET B). Specifically, I selected conference papers that meet both conditions: 1) share at least 1 author (with high similarity score) with the journal paper, 2) conference year is in the same year or earlier than the journal-paper year.
3. I evaluated each pair of journal papers and potential conference papers (coming from SET A and SET B), and identified all matching pairs (SET C).

<u><b>STEP 2 - Identify negative cases</u></b>
1. Those pairs that were identified as not matching were taken as negative cases (SET D)

<u><b>STEP 3 - Create golden dataset</u></b>
1. I combined SET C and SET D to create a golden dataset that contains both negative (xxx pairs) and positive pairs (xxx). 

Then, I randomly split the golden dataset into 2 sets: 1) train set and 2) test set

The train set contains 5,759 pairs: 353 positive, 5,406 negative


In [ ]:
train_path = f"{path_to_csv_dir}/training_labeled_210607.csv"
train_set = em.read_csv_metadata(
    train_path, 
    key='_id',
    ltable=JOUR, 
    rtable=CONF,
    fk_ltable='ltable_'+JOUR_INDEX, 
    fk_rtable='rtable_'+CONF_INDEX
)

train_set.gold_label.value_counts()

Metadata file is not present in the given path; proceeding to read the csv file.


0    5406
1     353
Name: gold_label, dtype: int64

C - Create feature vectors


Initialize Magellan with the list of features meta data...

In [ ]:
# Generate a set of features
feature_meta_data = em.get_features_for_matching(JOUR, CONF)
em.add_blackbox_feature(feature_meta_data, 'authors_similarity', authors_similarity)
em.add_blackbox_feature(feature_meta_data, 'first_author_similarity', first_author_similarity)
em.add_blackbox_feature(feature_meta_data, 'year_difference', year_difference)
em.add_blackbox_feature(feature_meta_data, 'bow_tfidf', bow_tfidf)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,id,id,numeric,short string (1 word),Not Applicable: Types do not match
1,year,year,numeric,numeric,Exact Match; Absolute Norm
2,bow,bow,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
3,authors,authors,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,authors_first_last,authors_first_last,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


Do you want to proceed? (y/n):y


True

In total, there are 29 features

In [ ]:
feature_meta_data.feature_name

0                                                 year_year_exm
1                                                 year_year_anm
2                                            year_year_lev_dist
3                                             year_year_lev_sim
4                                              bow_bow_lev_dist
5                                               bow_bow_lev_sim
6                                                   bow_bow_jar
7                                                   bow_bow_jwn
8                                                   bow_bow_exm
9                                       bow_bow_jac_qgm_3_qgm_3
10                              authors_authors_jac_qgm_3_qgm_3
11                          authors_authors_cos_dlm_dc0_dlm_dc0
12                          authors_authors_jac_dlm_dc0_dlm_dc0
13                                          authors_authors_mel
14                                     authors_authors_lev_dist
15                                      

Extract the feature vectors... (it will takes 5-6 hours) 

In [ ]:
# Convert the I into a set of feature vectors using features meta data.
feature_vectors = em.extract_feature_vecs(train_set, feature_table=F, attrs_after='gold_label')

In [ ]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(feature_vectors))

True

Populate (impute) missing features

In [ ]:
# Impute feature vectors with the mean of the column values.
em.set_key(feature_vectors, '_id') # key of the metadata
em.set_fk_ltable(feature_vectors, 'ltable_id') #foreign key to left table
em.set_fk_rtable(feature_vectors, 'rtable_id') #foreign key to right table
em.set_ltable(feature_vectors, JOUR) #Sets the ltable for a DataFrame in the catalog
em.set_rtable(feature_vectors, CONF) #Sets the rtable for a DataFrame in the catalog

True

In [ ]:
redudant_attrs = ['_id', 'ltable_'+JOUR_INDEX, 'rtable_'+CONF_INDEX, 'gold_label']
feature_vectors = em.impute_table(
    feature_vectors, 
    exclude_attrs=redudant_attrs, 
    missing_val=np.nan, 
    strategy='mean'
)

/usr/local/lib/python3.7/dist-packages/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


### 2.C. Train Matching Models

We try 6 matching algorithms for paper matching:
1. Decision Tree
2. Support Vector Machine
3. NaiveBayes
4. Random Forest
5. Logit regression
6. Linear Regression


In [ ]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
nb = em.SVMMatcher(name='NaiveBayes', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
algos = [dt, rf, svm, nb, ln, lg]

I select the best Machine Learning matcher based on average precision and average recall. In this study, I select the matcher that generates the highest the average recall score. This is because predicted pairs will be evaluated by an expert later to confirm that they are True Positive cases.

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher(
    algos, 
    table=feature_vectors, 
    exclude_attrs=redudant_attrs, 
    k=5, target_attr='gold_label', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.682491,0.668705,0.673889
1,RF,0.806462,0.749547,0.776919
2,SVM,0.942802,0.282329,0.433170
3,NaiveBayes,0.942802,0.282329,0.433170
4,LinReg,0.860427,0.630679,0.727728
5,LogReg,0.766283,0.577069,0.657309


I select random forest matcher because it has the highest score of average recall

In [ ]:
# Choose Random Forest to train
rf.fit(table=feature_vectors, exclude_attrs=['_id', 'gold_label'], target_attr='gold_label')

### 2.D. Evaluation

In [ ]:
test_meta_data = em.read_csv_metadata(f"{path_to_csv_dir}/testing_labeled_210607.csv", key='_id',
                                    ltable=JOUR, rtable=CONF,
                                    fk_ltable='ltable_'+JOUR_INDEX, fk_rtable='rtable_'+CONF_INDEX)

test_meta_data.dtypes

Metadata file is not present in the given path; proceeding to read the csv file.


_id                           int64
ltable_id                     int64
ltable_year                   int64
ltable_bow                   object
ltable_authors               object
ltable_authors_first_last    object
rtable_id                    object
rtable_year                   int64
rtable_bow                   object
rtable_authors               object
rtable_authors_first_last    object
rtable_first_author_first    object
rtable_first_author_last     object
gold_label                    int64
dtype: object

In [ ]:
# Convert the J into a set of feature vectors using F
test_feature_vectors = em.extract_feature_vecs(J, feature_table=F, attrs_after='gold_label')

In [ ]:
# Impute feature vectors with the mean of the column values.
em.set_key(test_feature_vectors, '_id') # key of the metadata
em.set_fk_ltable(test_feature_vectors, 'ltable_id') #foreign key to left table
em.set_fk_rtable(test_feature_vectors, 'rtable_id') #foreign key to right table
em.set_ltable(test_feature_vectors, JOUR) #Sets the ltable for a DataFrame in the catalog
em.set_rtable(test_feature_vectors, CONF) #Sets the rtable for a DataFrame in the catalog

True

In [ ]:
test_feature_vectors = em.impute_table(test_feature_vectors, exclude_attrs=['_id', 'gold_label'], missing_val=np.nan, strategy='mean')

/usr/local/lib/python3.7/dist-packages/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


In [ ]:
predictions = rf.predict(table=test_feature_vectors, exclude_attrs=['_id',  'gold_label'], append=True, target_attr='predicted', inplace=False)
predictions.to_csv(f"{path_to_csv_dir}/predictions_210607.csv", index=False)

In [ ]:
eval_summary = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_summary)

Precision : 82.11% (101/123)
Recall : 77.69% (101/130)
F1 : 79.84%
False positives : 22 (out of 123 positive predictions)
False negatives : 29 (out of 1816 negative predictions)


In [ ]:
test_feature_vectors.head()

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,bow_bow_lev_dist,bow_bow_lev_sim,bow_bow_jar,bow_bow_jwn,bow_bow_exm,bow_bow_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_jac_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,authors_authors_nmw,authors_authors_sw,authors_first_last_authors_first_last_jac_qgm_3_qgm_3,authors_first_last_authors_first_last_cos_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_jac_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_mel,authors_first_last_authors_first_last_lev_dist,authors_first_last_authors_first_last_lev_sim,authors_first_last_authors_first_last_nmw,authors_first_last_authors_first_last_sw,authors_similarity,first_author_similarity,year_difference,bow_tfidf,gold_label
0,0,13031.0,20049113.0,0.0,0.999501,1.0,0.75,904.0,0.303544,0.734995,0.734995,0.0,0.277019,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,33.0,33.0,1.000000,1.000000,1.0,1.000000,0.0,1.000000,31.0,31.0,15.0,-0.5,1.0,0.000000,0
1,1,14362.0,201218113.0,0.0,0.999503,1.0,0.75,990.0,0.281567,0.690533,0.690533,0.0,0.233796,0.025974,0.000000,0.000000,0.544786,35.0,0.125000,1.0,4.0,0.031250,0.000000,0.0,0.481332,30.0,0.210526,-4.0,4.0,-1.0,-1.0,1.0,0.000085,0
2,2,18612.0,201115511.0,0.0,0.997025,1.0,0.75,1134.0,0.273077,0.705764,0.735188,0.0,0.236811,0.052632,0.000000,0.000000,0.597098,34.0,0.170732,2.0,7.0,0.060606,0.000000,0.0,0.543474,30.0,0.117647,2.0,7.0,0.5,-1.0,6.0,0.000000,0
3,3,9287.0,201013981.0,0.0,0.999503,1.0,0.75,544.0,0.250689,0.765874,0.765874,0.0,0.286392,0.333333,0.288675,0.166667,0.458333,18.0,0.437500,-3.0,15.0,0.305556,0.333333,0.2,0.574603,19.0,0.321429,-4.0,9.0,9.0,-1.0,1.0,0.785191,0
4,4,1140.0,200710301.0,0.0,0.999004,1.0,0.75,1003.0,0.324579,0.785146,0.785146,0.0,0.511442,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,27.0,27.0,1.000000,1.000000,1.0,1.000000,0.0,1.000000,25.0,25.0,12.0,-1.0,2.0,0.145806,1


## 3 - ACTUAL MATCHING

In [ ]:
L = em.read_csv_metadata(f"{path_to_csv_dir}/paper_pairs_jour_conf_to_predict_210612.csv", key='_id', ltable=JOUR, rtable=CONF,
                                    fk_ltable='ltable_'+JOUR_INDEX, fk_rtable='rtable_'+CONF_INDEX)
print(L.index)
for i in range(1,63):
  idx_start = (i-1)*500
  idx_end = idx_start + 499
  # print(idx_start, idx_end)
  L.loc[idx_start:idx_end].to_csv(f"{path_to_csv_dir}/L/L{i}_210612.csv", index=0)
L.loc[31000:31285].to_csv(f"{path_to_csv_dir}/L/L63_210612.csv", index=0)

Metadata file is not present in the given path; proceeding to read the csv file.


RangeIndex(start=0, stop=31285, step=1)


In [ ]:
M = pd.DataFrame()
for i in range(1, 64):
  L = em.read_csv_metadata(f"{path_to_csv_dir}/L/L{i}_210612.csv", key='_id', ltable=JOUR, rtable=CONF, 
                           fk_ltable='ltable_'+JOUR_INDEX, fk_rtable='rtable_'+CONF_INDEX) 
  M = em.extract_feature_vecs(L, feature_table=F)
  M = M.append(data)
print(M.shape[0])
M.head()

31285


,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,bow_bow_lev_dist,bow_bow_lev_sim,bow_bow_jar,bow_bow_jwn,bow_bow_exm,bow_bow_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_jac_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,authors_authors_nmw,authors_authors_sw,authors_first_last_authors_first_last_jac_qgm_3_qgm_3,authors_first_last_authors_first_last_cos_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_jac_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_mel,authors_first_last_authors_first_last_lev_dist,authors_first_last_authors_first_last_lev_sim,authors_first_last_authors_first_last_nmw,authors_first_last_authors_first_last_sw,authors_similarity,first_author_similarity,year_difference,bow_tfidf
0,0,9440,2001_1001_5,0,0.993052,2.0,0.50,1020,0.315895,0.724455,0.724455,0,0.298469,0.287671,0.223607,0.125000,0.842564,26.0,0.500000,13.0,20.0,0.279412,0.250000,0.142857,0.841115,24.0,0.478261,13.0,20.0,9.500000,0.0,14,1.000000
1,1,9285,2001_1001_5,0,0.995522,2.0,0.50,755,0.248008,0.773021,0.773021,0,0.291304,0.339286,0.316228,0.166667,0.873077,33.0,0.365385,-14.0,19.0,0.360000,0.353553,0.200000,0.878261,28.0,0.391304,-10.0,18.0,18.000000,0.0,9,0.193163
2,2,5999,2001_1001_5,0,0.996514,1.0,0.75,710,0.292829,0.739999,0.739999,0,0.239354,0.223684,0.000000,0.000000,0.567568,44.0,0.153846,-7.0,19.0,0.231884,0.000000,0.000000,0.583040,39.0,0.152174,-4.0,18.0,8.500000,0.0,7,1.000000
3,3,955,2001_1001_5,0,0.997507,1.0,0.75,833,0.287425,0.779567,0.779567,0,0.309840,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,52.0,52.0,0.903846,0.894427,0.800000,0.987755,3.0,0.938776,43.0,43.0,13.666667,0.0,5,0.000056
4,4,6398,2001_1001_5,0,0.998005,1.0,0.75,722,0.280876,0.765160,0.765160,0,0.290323,0.466667,0.000000,0.000000,0.725962,37.0,0.288462,-6.0,19.0,0.481481,0.000000,0.000000,0.717391,34.0,0.260870,-7.0,18.0,14.500000,0.0,4,0.084176


In [ ]:
# Impute feature vectors with the mean of the column values.
em.set_key(M, '_id') # key of the metadata
em.set_fk_ltable(M, 'ltable_id') #foreign key to left table
em.set_fk_rtable(M, 'rtable_id') #foreign key to right table
em.set_ltable(M, JOUR) #Sets the ltable for a DataFrame in the catalog
em.set_rtable(M, CONF) #Sets the rtable for a DataFrame in the catalog

True

In [ ]:
N = em.impute_table(M, exclude_attrs=['_id'], missing_val=np.nan, strategy='mean')

/usr/local/lib/python3.7/dist-packages/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


In [ ]:
predictions = rf.predict(table=N, exclude_attrs=['_id'], append=True, target_attr='predicted', inplace=False)
predictions.to_csv(f"{path_to_csv_dir}/predictions_actual_matching_210607.csv", index=False)

In [ ]:
predictions.head()

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,bow_bow_lev_dist,bow_bow_lev_sim,bow_bow_jar,bow_bow_jwn,bow_bow_exm,bow_bow_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_jac_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,authors_authors_nmw,authors_authors_sw,authors_first_last_authors_first_last_jac_qgm_3_qgm_3,authors_first_last_authors_first_last_cos_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_jac_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_mel,authors_first_last_authors_first_last_lev_dist,authors_first_last_authors_first_last_lev_sim,authors_first_last_authors_first_last_nmw,authors_first_last_authors_first_last_sw,authors_similarity,first_author_similarity,year_difference,bow_tfidf,predicted
0,0,9440.0,200110015.0,0.0,0.993052,2.0,0.50,1020.0,0.315895,0.724455,0.724455,0.0,0.298469,0.287671,0.223607,0.125000,0.842564,26.0,0.500000,13.0,20.0,0.279412,0.250000,0.142857,0.841115,24.0,0.478261,13.0,20.0,9.500000,0.0,14.0,1.000000,0
1,1,9285.0,200110015.0,0.0,0.995522,2.0,0.50,755.0,0.248008,0.773021,0.773021,0.0,0.291304,0.339286,0.316228,0.166667,0.873077,33.0,0.365385,-14.0,19.0,0.360000,0.353553,0.200000,0.878261,28.0,0.391304,-10.0,18.0,18.000000,0.0,9.0,0.193163,0
2,2,5999.0,200110015.0,0.0,0.996514,1.0,0.75,710.0,0.292829,0.739999,0.739999,0.0,0.239354,0.223684,0.000000,0.000000,0.567568,44.0,0.153846,-7.0,19.0,0.231884,0.000000,0.000000,0.583040,39.0,0.152174,-4.0,18.0,8.500000,0.0,7.0,1.000000,0
3,3,955.0,200110015.0,0.0,0.997507,1.0,0.75,833.0,0.287425,0.779567,0.779567,0.0,0.309840,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,52.0,52.0,0.903846,0.894427,0.800000,0.987755,3.0,0.938776,43.0,43.0,13.666667,0.0,5.0,0.000056,0
4,4,6398.0,200110015.0,0.0,0.998005,1.0,0.75,722.0,0.280876,0.765160,0.765160,0.0,0.290323,0.466667,0.000000,0.000000,0.725962,37.0,0.288462,-6.0,19.0,0.481481,0.000000,0.000000,0.717391,34.0,0.260870,-7.0,18.0,14.500000,0.0,4.0,0.084176,0


In [ ]:
predictions.predicted.value_counts()

0    30106
1     1179
Name: predicted, dtype: int64

In [ ]:
prediction_pos = predictions[predictions.predicted==1][['_id']]
pos_id = list(prediction_pos['_id'])
# x = 332
# x in pos_id
data = M[M['_id'].isin(pos_id)]
data = data.drop_duplicates(['ltable_id', 'rtable_id'])
data.shape[0]

456

In [ ]:
data.head()

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,bow_bow_lev_dist,bow_bow_lev_sim,bow_bow_jar,bow_bow_jwn,bow_bow_exm,bow_bow_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_jac_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,authors_authors_nmw,authors_authors_sw,authors_first_last_authors_first_last_jac_qgm_3_qgm_3,authors_first_last_authors_first_last_cos_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_jac_dlm_dc0_dlm_dc0,authors_first_last_authors_first_last_mel,authors_first_last_authors_first_last_lev_dist,authors_first_last_authors_first_last_lev_sim,authors_first_last_authors_first_last_nmw,authors_first_last_authors_first_last_sw,authors_similarity,first_author_similarity,year_difference,bow_tfidf
214,332,18468,2001_1017_3,1,1.000000,0.0,1.00,1237,0.310864,0.736812,0.842087,0,0.363351,0.371795,0.353553,0.200000,0.848987,42.0,0.432432,-10.0,24.0,0.406250,0.447214,0.285714,0.861790,33.0,0.431034,-3.0,24.0,11.500000,2.0,0,0.215363
225,348,5441,2001_1018_3,0,0.997010,1.0,0.75,1155,0.312500,0.739044,0.739044,0,0.443418,0.783333,0.547723,0.375000,0.917113,6.0,0.892857,47.0,47.0,0.741379,0.612372,0.428571,0.889160,9.0,0.830189,37.0,37.0,11.833333,0.5,6,0.988601
378,621,19384,2001_1021_1,0,0.998005,1.0,0.75,710,0.323165,0.752832,0.752832,0,0.362984,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,35.0,35.0,0.861111,0.866025,0.750000,0.981818,3.0,0.909091,27.0,27.0,13.000000,-1.0,4,1.000000
451,744,9930,2001_1026_2,0,0.998005,1.0,0.75,484,0.486744,0.778920,0.778920,0,0.505976,0.886364,0.800000,0.666667,0.948846,1.0,0.975000,38.0,38.0,0.761905,0.774597,0.600000,0.912171,6.0,0.842105,26.0,26.0,12.500000,-1.0,4,0.000000
167,1074,15987,2001_1035_2,0,0.999001,1.0,0.75,259,0.581583,0.783514,0.870108,0,0.643032,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,60.0,60.0,0.912281,0.894427,0.800000,0.911696,3.0,0.947368,51.0,51.0,16.333333,1.0,2,0.995115


In [ ]:
POS = L[L._id.isin(data._id)]
POS.head(n=1)

,_id,ltable_id,ltable_year,ltable_bow,ltable_authors,ltable_authors_first_last,rtable_id,rtable_year,rtable_bow,rtable_authors,rtable_authors_first_last,rtable_first_author_first,rtable_first_author_last
214,332,18468,2001,"top,management,team,heterogeneity,personality,power,and,proxies,this,article,reports,partial,res...","Pitcher, Patricia|Smith, Anne D.",Patricia Pitcher|Anne D. Smith,2001_1017_3,2001,"top,management,team,heterogeneity,power,and,performance,in,a,hospital,setting,top,management,tea...","Pitcher, Patricia Cherie|Smith, Anne D|Hood, Jacqueline N.|Houghton, Susan",Patricia Pitcher|Anne Smith|Jacqueline Hood|Susan Houghton,Patricia,Pitcher


In [ ]:
POS['ltable_numau'] = 0
POS['rtable_numau'] = 0

for idx, row in POS.iterrows():
  POS.at[idx, 'ltable_numau'] = len(row.ltable_authors.split("|"))
  POS.at[idx, 'rtable_numau'] = len(row.rtable_authors.split("|"))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
POS.iloc[0]

_id                                                                                                                          332
ltable_id                                                                                                                  18468
ltable_year                                                                                                                 2001
ltable_bow                   top,management,team,heterogeneity,personality,power,and,proxies,this,article,reports,partial,res...
ltable_authors                                                                                  Pitcher, Patricia|Smith, Anne D.
ltable_authors_first_last                                                                         Patricia Pitcher|Anne D. Smith
rtable_id                                                                                                            2001_1017_3
rtable_year                                                                                      

In [ ]:
POS['match22'] = (POS.ltable_numau == POS.rtable_numau) & (POS.ltable_numau == 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
POS['year_diff'] = POS.ltable_year - POS.rtable_year 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
POS.iloc[0]

_id                                                                                                                          332
ltable_id                                                                                                                  18468
ltable_year                                                                                                                 2001
ltable_bow                   top,management,team,heterogeneity,personality,power,and,proxies,this,article,reports,partial,res...
ltable_authors                                                                                  Pitcher, Patricia|Smith, Anne D.
ltable_authors_first_last                                                                         Patricia Pitcher|Anne D. Smith
rtable_id                                                                                                            2001_1017_3
rtable_year                                                                                      

In [ ]:
POS.to_csv(f"{path_to_csv_dir}/predictions_actual_matching_pos_210607.csv", index=False)